In [293]:
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB

### data

In [294]:
alpha = pd.read_csv("result_alpha.csv").to_numpy().flatten()
beta_plus = pd.read_csv("result_beta_plus.csv").to_numpy()
beta_minus = pd.read_csv("result_beta_minus.csv").to_numpy()

R_flat = pd.read_csv("result_R.csv")

I_len = R_flat["generator"].nunique()
T_len = R_flat["time"].nunique() 
S_len = R_flat["scenario"].nunique()

R = R_flat.pivot_table(
    values="value",
    index="generator",
    columns=["time", "scenario"]
).to_numpy()
R = R.reshape(I_len, T_len, S_len)

### commitment model

In [295]:
I, T, S = R.shape
com = gp.Model("Commitment")
com.Params.OutputFlag = 0

x = com.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
y_plus = com.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
y_minus = com.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
M = max(sum(R[i, t, s] for i in range(I)) for t in range(T) for s in range(S))
z = com.addVars(I, T, S, vtype=GRB.BINARY, name="z")

com.update()

In [296]:
# x 값들 간 최대 차이를 최소화
obj = gp.quicksum((x[i, t] - alpha[t] / I)**2 for i in range(I) for t in range(T))
com.setObjective(obj, GRB.MINIMIZE)

In [297]:
# x로 y+, y- 결정 지어주기
for i in range(I):
    for t in range(T):
        for s in range(S):
            com.addConstr(R[i, t, s] - x[i, t] == y_plus[i, t, s] - y_minus[i, t, s])

# real-time market 참여 제약
for i in range(I):
    for t in range(T):
        for s in range(S):
            com.addConstr(R[i,t,s] - y_plus[i, t, s] >= 0)

# y+, y- 둘 중 하나는 0이어야함
for i in range(I):
    for t in range(T):
        for s in range(S):
            com.addConstr(y_plus[i, t, s] <= M * z[i, t, s])
            com.addConstr(y_minus[i, t, s] <= M * (1 - z[i, t, s]))

# alpha는 모든 x의 합이다
for t in range(T):
    com.addConstr(
        gp.quicksum(x[i, t] for i in range(I)) == alpha[t]
    )

com.optimize()

if com.status == GRB.OPTIMAL:
    x_vals = np.array([[x[i, t].x for t in range(T)] for i in range(I)])
    y_plus_vals = np.array([[[y_plus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)])
    y_minus_vals = np.array([[[y_minus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)])

    print("Optimal solution found!")
    print(com.objVal)
    # print("x (commitment):", x_vals)
    # print("y+ (excess):", y_plus_vals)
    # print("y- (shortage):", y_minus_vals)
else:
    print("\nNo optimal solution found.")

Optimal solution found!
1.7053025658242404e-13


In [298]:
for t in range(T):
    for i in range(I):
        print(f"t={t} i={i}")
        print(f"x={x_vals[i,t]:.4f}")
        print(f"y+=", end=" ")
        for s in range(S):
            print(f"{y_plus_vals[i,t,s]:.4f}", end=" ")
        print("\ny-=", end=" ")
        for s in range(S):
            print(f"{y_minus_vals[i,t,s]:.4f}", end=" ")
        print("\nR=", end=" ")
        for s in range(S):
            print(f"{R[i,t,s]:.4f}", end=" ")
        print("\n")

t=0 i=0
x=0.0000
y+= 0.0000 0.0000 0.0000 
y-= 0.0000 0.0000 0.0000 
R= 0.0000 0.0000 0.0000 

t=0 i=1
x=0.0000
y+= 0.0000 0.0000 0.0000 
y-= 0.0000 0.0000 0.0000 
R= 0.0000 0.0000 0.0000 

t=0 i=2
x=0.0000
y+= 0.0000 0.0000 0.0000 
y-= 0.0000 0.0000 0.0000 
R= 0.0000 0.0000 0.0000 

t=0 i=3
x=0.0000
y+= 0.0000 0.0000 0.0000 
y-= 0.0000 0.0000 0.0000 
R= 0.0000 0.0000 0.0000 

t=0 i=4
x=0.0000
y+= 0.0000 0.0000 0.0000 
y-= 0.0000 0.0000 0.0000 
R= 0.0000 0.0000 0.0000 

t=0 i=5
x=0.0000
y+= 0.0000 0.0000 0.0000 
y-= 0.0000 0.0000 0.0000 
R= 0.0000 0.0000 0.0000 

t=1 i=0
x=0.0000
y+= 0.0000 0.0000 0.0000 
y-= 0.0000 0.0000 0.0000 
R= 0.0000 0.0000 0.0000 

t=1 i=1
x=0.0000
y+= 0.0000 0.0000 0.0000 
y-= 0.0000 0.0000 0.0000 
R= 0.0000 0.0000 0.0000 

t=1 i=2
x=0.0000
y+= 0.0000 0.0000 0.0000 
y-= 0.0000 0.0000 0.0000 
R= 0.0000 0.0000 0.0000 

t=1 i=3
x=0.0000
y+= 0.0000 0.0000 0.0000 
y-= 0.0000 0.0000 0.0000 
R= 0.0000 0.0000 0.0000 

t=1 i=4
x=0.0000
y+= 0.0000 0.0000 0.0000 
y-= 0.0

In [299]:
for t in range(12, 15):
    x_sum = sum(x_vals[i, t] for i in range(I))
    for s in range(S):
        y_plus_sum = sum(y_plus_vals[i, t, s] for i in range(I))
        y_minus_sum = sum(y_minus_vals[i, t, s] for i in range(I))
        print(f"[t={t} s={s}]")
        print(f"alpha: {alpha[t]:.2f}")
        print(f"beta+: {beta_plus[t,s]:.2f}")
        print(f"beta-: {beta_minus[t,s]:.2f}")
        print(f"x 합계: {x_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {x_vals[i,t]:.2f}", end=" ")
        print(")")
        print(f"y+ 합계: {y_plus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {y_plus_vals[i,t,s]:.2f}", end=" ")
        print(")")
        print(f"y- 합계: {y_minus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {y_minus_vals[i,t,s]:.2f}", end=" ")
        print(")")
        print()

[t=12 s=0]
alpha: 17.69
beta+: 10.17
beta-: 0.00
x 합계: 17.69 ([0] 2.95 [1] 2.95 [2] 2.95 [3] 2.95 [4] 2.95 [5] 2.95 )
y+ 합계: 16.07 ([0] 6.43 [1] 0.00 [2] 4.98 [3] 3.07 [4] 0.00 [5] 1.59 )
y- 합계: 5.90 ([0] 0.00 [1] 2.95 [2] 0.00 [3] 0.00 [4] 2.95 [5] 0.00 )

[t=12 s=1]
alpha: 17.69
beta+: 0.00
beta-: 0.00
x 합계: 17.69 ([0] 2.95 [1] 2.95 [2] 2.95 [3] 2.95 [4] 2.95 [5] 2.95 )
y+ 합계: 5.95 ([0] 3.77 [1] 0.00 [2] 1.91 [3] 0.00 [4] 0.28 [5] 0.00 )
y- 합계: 5.95 ([0] 0.00 [1] 1.27 [2] 0.00 [3] 1.73 [4] 0.00 [5] 2.95 )

[t=12 s=2]
alpha: 17.69
beta+: 10.05
beta-: 0.00
x 합계: 17.69 ([0] 2.95 [1] 2.95 [2] 2.95 [3] 2.95 [4] 2.95 [5] 2.95 )
y+ 합계: 14.70 ([0] 5.24 [1] 1.42 [2] 8.05 [3] 0.00 [4] 0.00 [5] 0.00 )
y- 합계: 4.66 ([0] 0.00 [1] 0.00 [2] 0.00 [3] 1.89 [4] 1.90 [5] 0.86 )

[t=13 s=0]
alpha: 28.89
beta+: 0.00
beta-: 0.00
x 합계: 28.89 ([0] 4.82 [1] 4.82 [2] 4.82 [3] 4.82 [4] 4.82 [5] 4.82 )
y+ 합계: 11.27 ([0] 0.00 [1] 0.00 [2] 0.14 [3] 5.82 [4] 0.00 [5] 5.31 )
y- 합계: 11.27 ([0] 3.25 [1] 4.82 [2] 0.00 

In [300]:
# 베타...........
# 목적함수 값 비교